In [31]:
import tensorflow as tf
import bentoml
import PIL.Image as Image
import numpy as np

In [32]:
IMAGE_SHAPE = (224, 224)
user_input_image = tf.keras.utils.get_file('image.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/grace_hopper.jpg')
user_input_image = Image.open(user_input_image).resize(IMAGE_SHAPE)
user_input_image = np.array(user_input_image)/255.0
predict_data = user_input_image[np.newaxis, ...].tolist()

/tmp/ipykernel_2004/3638141814.py:5: RuntimeWarning: coroutine 'main' was never awaited
  predict_data = user_input_image[np.newaxis, ...].tolist()


In [33]:
mobilenet_runner = bentoml.tensorflow.get("mobilenet:0.1").to_runner()
mobilenet_runner.init_local()
result = mobilenet_runner.run(predict_data)
print(result)

'Runner.init_local' is for debugging and testing only. Make sure to remove it before deploying to production.
/home/user/anaconda3/envs/bentoml/lib/python3.10/typing.py:1415: RuntimeWarning: coroutine 'main' was never awaited
  for base in cls.__mro__[:-1]:  # without object


[[ 0.19967595  1.0678965  -0.09609853 ...  0.10602828 -0.56032073
  -0.07089337]]


In [34]:
from load_model import postprocessing

In [35]:
predicted_class = np.argmax(result[0], axis=-1)
predicted_class_name = postprocessing(predicted_class)
result = {"predicted":str(result[0]), "class":predicted_class_name}
print(result)

{'predicted': '[ 0.19967595  1.0678965  -0.09609853 ...  0.10602828 -0.56032073\n -0.07089337]', 'class': 'military uniform'}


In [ ]:
IMAGE_SHAPE = (224, 224)
user_input_image = tf.keras.utils.get_file('image.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/grace_hopper.jpg')
user_input_image = Image.open(user_input_image).resize(IMAGE_SHAPE)
user_input_image = np.array(user_input_image)/255.0
predict_data = user_input_image[np.newaxis, ...].tolist()

mobilenet_runner = bentoml.tensorflow.get("mobilenet:0.1").to_runner()
mobilenet_runner.init_local()
result = mobilenet_runner.run(predict_data)
print(result)

from load_model import postprocessing
predicted_class = np.argmax(result[0], axis=-1)
predicted_class_name = postprocessing(predicted_class)
result = {"predicted":str(result[0]), "class":predicted_class_name}
print(result)

In [46]:
%%writefile service.py
import tensorflow as tf
import bentoml
from PIL.Image import Image as PILImage
import numpy as np
import base64

# from bentoml.io import NumpyNdarray
from bentoml.io import JSON
from bentoml.io import Image



mobilenet_runner = bentoml.tensorflow.get("mobilenet:0.1").to_runner()
# mobilenet_runner.init_local()
svc = bentoml.Service( name="mobilenet", runners=[mobilenet_runner] )

def postprocessing(predicted_class):
    imagenet_labels = np.array(open('./ImageNetLabels.txt').read().splitlines())
    predicted_class_name = imagenet_labels[predicted_class]
    return predicted_class_name


@svc.api(input=Image(), output=JSON())
async def predict(f: PILImage) -> "JSON":
    assert isinstance(f, PILImage)
    IMAGE_SHAPE = (224, 224)
    image = np.array(f.resize(IMAGE_SHAPE)).astype("float32")
    user_input_image = f.resize(IMAGE_SHAPE)
    user_input_image = np.array(user_input_image)/255.0
    predict_data = user_input_image[np.newaxis, ...].tolist()
    result = await mobilenet_runner.async_run(predict_data) # 추론하는 code 
    predicted_class = np.argmax(result[0], axis=-1) # 가장 높은 확률 출력 
    class_name = postprocessing(predicted_class) # class 로 변경 
    return class_name

Overwriting service.py


In [47]:
!bentoml serve service.py:svc --reload -p 5002

2023-03-16 10:56:19.332474: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-16 10:56:20.514553: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-16 10:56:20.514659: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-16 10:56:20.514698: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv